<a href="https://colab.research.google.com/github/StevenMorlier/Colab-LLM-Connect/blob/main/ollama_ngrok_cline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows how to install and run ollama on google colab + tunneling using ngrok.

set `NGROK_TOKEN` in colab secrets panel.

Gholamreza Dar 2024

https://stackoverflow.com/questions/77697302/how-to-run-ollama-in-google-colab

template pour cline :
https://github.com/maryasov/ollama-models-instruct-for-cline.git

definir le modele choisi

In [ ]:
import re

# Définir le modèle de base ICI
#MODEL_NAME = "qwen2.5-coder:7b" # <---
MODEL_NAME = "llama3.1:8b" # <---
MODEL_NAME = "deepseek-r1:14b" # <---


# Créer automatiquement le nom du modèle avec template en remplaçant ce qui est après les ":"
MODEL_WITH_TEMPLATE = re.sub(r'(.*:).*', r'\1cline', MODEL_NAME)

print(f"Modèle de base: {MODEL_NAME}")
print(f"Modèle avec template: {MODEL_WITH_TEMPLATE}")

## test GPU

In [ ]:
!apt-get install -y lshw
!apt-get install -y pciutils
!nvcc --version
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

  from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('❌ Not using a high-RAM runtime')
else:
  print('✅ You are using a high-RAM runtime!')

In [ ]:
# Installing Ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
# needed for tunneling via ngrok
%env OLLAMA_HOST=0.0.0.0
# Running Ollama and serving on localhost:11434
!nohup ollama serve &
!ollama pull $MODEL_NAME

In [ ]:
import re
import subprocess
import os

# Variables globales pour les noms des modèles
#MODEL_NAME = "qwen2.5-coder:32b"  # Modèle de base
#MODEL_WITH_TEMPLATE = re.sub(r'(.*:).*', r'\1cline', MODEL_NAME)  # Modèle avec template

def create_cline_modelfile(
    base_model=MODEL_NAME,
    model_name=MODEL_WITH_TEMPLATE,
    modelfile_path=None,
    create_model=True,
    parameters={
        "num_ctx": 32768,
        "temperature": 0.1,
        "top_p": 0.7,
        "presence_penalty": 1.0,
        "frequency_penalty": 1.0,
        "stop": ["<|im_start|>", "<|im_end|>"]  # Ajout des deux valeurs d'arrêt
    }
):
    """
    Crée un Modelfile pour Ollama avec le template Cline et crée optionnellement le modèle.
    Utilise les variables globales MODEL_NAME et MODEL_WITH_TEMPLATE par défaut.

    Args:
        base_model: Modèle de base à utiliser (par défaut: MODEL_NAME)
        model_name: Nom du modèle à créer (par défaut: MODEL_WITH_TEMPLATE)
        modelfile_path: Chemin où créer le Modelfile (si None, dérivé du nom du modèle)
        create_model: Si True, exécute la commande ollama create
        parameters: Dictionnaire des paramètres à utiliser pour le modèle

    Returns:
        str: Chemin du Modelfile créé
    """
    # Si modelfile_path n'est pas spécifié, le dériver du nom du modèle
    if modelfile_path is None:
        model_short_name = model_name.split(':')[0]
        modelfile_path = f"Modelfile.{model_short_name}"

    # Template Cline avancé avec support des outils
    template_content = """{{- /* Initial system message with core instructions */ -}}
{{- if .Messages }}
{{- if or .System .Tools }}
<|im_start|>system
{{- if .System }}
{{ .System }}
{{- end }}

{{- if .Tools }}
# Tools and XML Schema
You have access to the following tools. Each tool must be used according to this XML schema:

<tools>
{{- range .Tools }}
{{ .Function }}
{{- end }}
</tools>

## Tool Use Format
1. Think about the approach in <thinking> tags
2. Call tool using XML format:
   <tool_name>
     <param_name>value</param_name>
   </tool_name>
3. Process tool response from:
   <tool_response>result</tool_response>
{{- end }}
<|im_end|>
{{- end }}

{{- /* Message handling loop */ -}}
{{- range $i, $_ := .Messages }}
{{- $last := eq (len (slice $.Messages $i)) 1 }}

{{- /* User messages */ -}}
{{- if eq .Role "user" }}
<|im_start|>user
{{ .Content }}
<|im_end|>

{{- /* Assistant messages */ -}}
{{- else if eq .Role "assistant" }}
<|im_start|>assistant
{{- if .Content }}
{{ .Content }}
{{- else if .ToolCalls }}
{{- range .ToolCalls }}
<thinking>
[Analysis of current state and next steps]
</thinking>

<{{ .Function.Name }}>
{{- range $key, $value := .Function.Arguments }}
<{{ $key }}>{{ $value }}</{{ $key }}>
{{- end }}
</{{ .Function.Name }}>
{{- end }}
{{- end }}
<|im_end|>

{{- /* Tool response handling */ -}}
{{- else if eq .Role "tool" }}
<|im_start|>tool
<tool_response>
{{ .Content }}
</tool_response>
<|im_end|>
{{- end }}

{{- /* Prepare for next assistant response if needed */ -}}
{{- if and (ne .Role "assistant") $last }}
<|im_start|>assistant
{{- end }}
{{- end }}

{{- /* Handle single message case */ -}}
{{- else }}
{{- if .System }}
<|im_start|>system
{{ .System }}
<|im_end|>
{{- end }}

{{- if .Prompt }}
<|im_start|>user
{{ .Prompt }}
<|im_end|>
{{- end }}
<|im_start|>assistant
{{- end }}
{{ .Response }}<|im_end|>"""

    # Créer le Modelfile
    with open(modelfile_path, 'w') as f:
        # La ligne FROM est obligatoire
        f.write(f'FROM {base_model}\n\n')

        # Commencer la définition du template
        f.write('TEMPLATE """\n')
        f.write(template_content + '\n"""\n\n')

        # Ajouter les paramètres
        for param_name, param_value in parameters.items():
            # Gérer les cas spéciaux qui nécessitent des guillemets
            if param_name == "stop":
                if isinstance(param_value, list):
                    for stop_value in param_value:
                        f.write(f'PARAMETER stop "{stop_value}"\n')
                else:
                    f.write(f'PARAMETER stop "{param_value}"\n')
            else:
                f.write(f'PARAMETER {param_name} {param_value}\n')

    print(f"✅ Modelfile créé: {modelfile_path}")
    print(f"   Base model: {base_model}")
    print(f"   Template model: {model_name}")

    # Afficher le contenu du Modelfile (optionnel)
    # print("\n--- Contenu du Modelfile ---")
    # with open(modelfile_path, 'r') as f:
    #     print(f.read())
    # print("---------------------------\n")

    # Créer le modèle si demandé
    if create_model:
        print(f"🔄 Création du modèle {model_name}...")
        cmd = f"ollama create {model_name} -f {modelfile_path}"
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

        if result.returncode == 0:
            print(f"✅ Modèle {model_name} créé avec succès!")
        else:
            print(f"❌ Erreur lors de la création du modèle: {result.stderr}")

    return modelfile_path

# Fonction pour utiliser le modèle (avec ou sans template)
def generate_with_model(prompt, use_template=True, temperature=0.1):
    """
    Génère du texte en utilisant soit le modèle de base, soit le modèle avec template

    Args:
        prompt: Le prompt à envoyer au modèle
        use_template: Si True, utilise MODEL_WITH_TEMPLATE, sinon MODEL_NAME
        temperature: Température pour la génération

    Returns:
        str: La réponse du modèle
    """
    import json

    model = MODEL_WITH_TEMPLATE if use_template else MODEL_NAME

    data = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature
    }

    cmd = f'''
    curl http://localhost:11434/api/generate -d '{json.dumps(data)}' -s
    '''

    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return result.stdout

# Exemple d'utilisation
if __name__ == "__main__":
    print(f"Modèle de base: {MODEL_NAME}")
    print(f"Modèle avec template: {MODEL_WITH_TEMPLATE}")

    # Créer le modèle avec template
    create_cline_modelfile()

    # Exemple d'utilisation des modèles
    #print("\nTest du modèle de base:")
    #response = generate_with_model("Écris une fonction Python qui calcule la factorielle", use_template=False)
    #print(response)

    #print("\nTest du modèle avec template:")
    #response = generate_with_model("Écris une fonction Python qui calcule la factorielle", use_template=True)
    #print(response)
    # Check the available models
!ollama list

## Curl

In [ ]:
# Test the API using curl locally with the MODEL_WITH_TEMPLATE variable
import json

# Préparation des données avec la variable MODEL_WITH_TEMPLATE
data = {
    "model": MODEL_WITH_TEMPLATE,
    "prompt": "Question: Who was the first president of the United States? \n Only answer using a few words. maybe just a name Answer: "
}

# Construction de la commande curl avec les données JSON
!curl http://localhost:11434/api/generate -d '{json.dumps(data)}'

## Expose the API publicly using ngrok

In [ ]:
!pip install -qU pyngrok

In [ ]:
from google.colab import userdata
from pyngrok import ngrok, conf

# get NGROK_TOKEN from colab secrets
ngrok_token = userdata.get('NGROK_TOKEN')
if not ngrok_token:
    raise ValueError("NGROK_TOKEN secret not found. Please add it to Colab secrets.")

# Set the ngrok auth token using Python
conf.get_default().auth_token = ngrok_token
ngrok.set_auth_token(ngrok_token)

# Expose Ollama server via ngrok on port 11434
public_url = ngrok.connect("http://localhost:11434")
print(f"Ollama server public URL for CLINE: {public_url.public_url}")
print(f"Model: {MODEL_WITH_TEMPLATE} (do NOT add / at the end in cline url)")
print(f"commande pour utiliser sur ollama en local")
print(f"export OLLAMA_HOST={public_url.public_url}/")

In [ ]:
# Sleep for a few seconds.
import time
time.sleep(2)

# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')